In [4]:
import numpy as np
import MDSplus as mds
import retention as rt

MdsIpException: %MDSPLUS-E-Unknown, Error connecting to mds.ipp.ac.cn

In [ ]:
cn = mds.Connection('mds.ipp.ac.cn')
shot = 100000

In [4]:
cn.openTree('energy_east',shot)
tau_e = cn.get('\\taue_mhd').data()
time = cn.get('dim_of(taue_mhd)').data()

In [5]:
time

array([0.00000e+00, 1.00000e-04, 2.00000e-04, ..., 1.11978e+01,
       1.11979e+01, 1.11980e+01])

In [ ]:
def get_rglobal(shot):

    S_gas = get_Sgas(shot)
    tau_p = 2 * get_taue(shot)
    Ne = rt.get_plasma(shot)
    Ne = np.diff(Ne)
    R_global = S_gas(shot) * tau_p[1:]
    idx = Ne!=0
    R_global[idx] /= Ne[idx]

    return R_global

In [ ]:
def get_Sgas(shot):
    rate = [0.1,0.25,1] # gas puffing, SMBI, PI
    gas_puffing = rt.get_puff(shot)
    SMBI = rt.get_SMBI(shot)
    PI = rt.get_pi(shot)
    S_gas = np.diff(gas_puffing)*rate[0]+np.diff(SMBI)*rate[1]+np.diff(PI)*rate[2]
    return S_gas

In [ ]:
def get_taue(shot):
    (idx_begin,idx_end), time_ip = rt.get_time(shot)
    cn.openTree('energy_east',shot)
    tau_e = cn.get('\\taue_mhd').data()
    tau_e = tau_e[::10]
    time_taue = cn.get("dim_of(taue_mhd)").data()[::10]
    idx_tau_begin =  1000
    idx_tau_end = int(time_taue[-1]*1000)  + 1000
    # 统一频率
    tau_e_long = np.zeros(idx_end-idx_begin)
    tau_e_long[idx_tau_begin:idx_tau_end] = tau_e
    return tau_e_long